# Colorization


In [1]:
import cv2
import numpy as np

prototxt = "Colorize/colorization_deploy_v2.prototxt"
caffe_model = "Colorize/colorization_release_v2.caffemodel"
pts_npy = "Colorize/pts_in_hull.npy"

# Loading the neural network that will colorize the video
net = cv2.dnn.readNetFromCaffe(prototxt, caffe_model)
pts = np.load(pts_npy)
 
layer1 = net.getLayerId("class8_ab")
layer2 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(layer1).blobs = [pts.astype("float32")]
net.getLayer(layer2).blobs = [np.full([1, 313], 2.606, dtype="float32")]

# Test on a sample video
cap = cv2.VideoCapture("sample.mp4")
out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 25, (1920,1080))

pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
while True:
    flag, frame = cap.read()
    if flag:

        pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        print(str(pos_frame)+" frames")
        # The frame is ready and already captured
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        scaled = frame.astype("float32") / 255.0
        lab = cv2.cvtColor(scaled, cv2.COLOR_RGB2LAB)

        # Resizing the image
        resized = cv2.resize(lab, (224, 224))
        # Extracting the value of L for LAB image
        L = cv2.split(resized)[0]
        L -= 50

        # Predicting a and b values
        # Setting input
        net.setInput(cv2.dnn.blobFromImage(L))
        # Finding the values of 'a' and 'b'
        ab = net.forward()[0, :, :, :].transpose((1, 2, 0))
        # Resizing
        ab = cv2.resize(ab, (frame.shape[1], frame.shape[0]))
        L = cv2.split(lab)[0]
        # Combining L,a,b
        LAB_colored = np.concatenate((L[:, :, np.newaxis], ab), axis=2)
        
        # Post-process the colorized image
        colorized = cv2.cvtColor(LAB_colored, cv2.COLOR_LAB2BGR)
        colorized = np.clip(colorized, 0, 1)
        colorized = (255 * colorized).astype("uint8")

        out.write(colorized)


    else:
        # The next frame is not ready, so we try to read it again
        cap.set(cv2.CAP_PROP_POS_FRAMES, pos_frame-1)
        print("frame is not ready")
        # It is better to wait for a while for the next frame to be ready
        cv2.waitKey(1000)

    if cv2.waitKey(10) == 27:
        break
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        # If the number of captured frames is equal to the total number of frames, we stop
        break
    
cap.release()
out.release()

1.0 frames
2.0 frames
3.0 frames
4.0 frames
5.0 frames
6.0 frames
7.0 frames
8.0 frames
9.0 frames
10.0 frames
11.0 frames
12.0 frames
13.0 frames
14.0 frames
15.0 frames
16.0 frames
17.0 frames
18.0 frames
19.0 frames
20.0 frames
21.0 frames
22.0 frames
23.0 frames
24.0 frames
25.0 frames
26.0 frames
27.0 frames
28.0 frames
29.0 frames
30.0 frames
31.0 frames
32.0 frames
33.0 frames
34.0 frames
35.0 frames
36.0 frames
37.0 frames
38.0 frames
39.0 frames
40.0 frames
41.0 frames
42.0 frames
43.0 frames
44.0 frames
45.0 frames
46.0 frames
47.0 frames
48.0 frames
49.0 frames
50.0 frames
51.0 frames
52.0 frames
53.0 frames
54.0 frames
55.0 frames
56.0 frames
57.0 frames
58.0 frames
59.0 frames
60.0 frames
61.0 frames
62.0 frames
63.0 frames
64.0 frames
65.0 frames
66.0 frames
67.0 frames
68.0 frames
69.0 frames
70.0 frames
71.0 frames
72.0 frames
73.0 frames
74.0 frames
75.0 frames
76.0 frames
77.0 frames
78.0 frames
79.0 frames
80.0 frames
81.0 frames
82.0 frames
83.0 frames
84.0 frames
8